In [1]:
# imports 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,precision_score ,recall_score ,f1_score
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
import tensorflow as tf 
from tensorflow import keras

In [1]:
# preprocessing
SAMPLE =2000
radius =(np.random.rand(SAMPLE) *100).reshape(-1,1)
area =(np.random.rand(SAMPLE) *100).reshape(-1,1)
premeter =(np.random.normal(loc =500.0 ,scale =100.0 ,size =SAMPLE)).reshape(-1,1)
texture =(np.random.normal(loc =5 ,scale =1.5 ,size =SAMPLE)).reshape(-1,1)
diagnosis =[]
true_count =0
false_count =0
diagnosis_count =[]
for i in range(SAMPLE) : 
    if radius[i ,0] >=30.0 and area[i ,0] >=30.0 and premeter[i ,0] >=300.0 and texture[i ,0] >=3.0 : 
        diagnosis.append(1)
        true_count +=1
    else :
        diagnosis.append(0)
        false_count +=1
diagnosis_count.append(true_count)
diagnosis_count.append(false_count)
diagnosis =np.asarray(diagnosis ,dtype =np.int32).reshape(-1,1)
data =np.concatenate((radius ,area ,premeter ,texture) ,axis =1)
data_norm =minmax_scale(data ,feature_range=(0,1)).astype(np.float32)
xtr ,xte ,ytr ,yte =train_test_split(data ,diagnosis ,train_size=0.8)
colors =['coral' ,'cyan'] 
labels =['malignant' ,'benign']
explods =[0.1 ,0.0]
font_title ={'family': 'serif',
        'color':  'darkred',
        'weight': 'bold',
        'size': 20,
        }
fig =plt.figure(num =1 ,dpi =100 ,figsize =(10,10))
ax1 =fig.add_subplot(1,1,1)
ax1.pie(diagnosis_count ,
        colors =colors ,
        labels =labels ,
        explode =explods ,
        startangle =45 ,
        autopct ='%1.1f%%' ,
        pctdistance =0.5 ,
        labeldistance =1.2)
plt.title(label ='diagnosis pie chart' ,pad =20 ,fontdict =font_title)
plt.legend(title ='diagnosis :')
plt.show()

In [2]:
# model creation 
def create_model(a_function ,h_unit ,l_rate) : 
  model =keras.Sequential()
  model.add(keras.layers.Dense(units =h_unit ,input_dim =4))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation(a_function))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(units =h_unit))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation(a_function))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(units =h_unit))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation(a_function))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(units =h_unit))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation(a_function))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(units =h_unit))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation(a_function))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(units =1))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('sigmoid'))
  model.compile(optimizer =keras.optimizers.Adam(learning_rate =l_rate) ,
                loss ='binary_crossentropy' ,
                metrics =['binary_accuracy'])
  return model

In [3]:
# random search cross validation 
cls = KerasClassifier(create_model ,
                    optimizer =keras.optimizers.Adam(learning_rate=0.0001) ,
                    loss =keras.losses.BinaryCrossentropy ,
                    metrics =[keras.metrics.BinaryAccuracy] ,
                    epochs =1 ,
                    verbose =2
)
params ={ 
    'model__a_function' : ['relu' ,'tanh'] ,
    'model__h_unit' : [256 ,512] ,
    'model__l_rate' : [0.0003 ,0.0005]
}
rs =RandomizedSearchCV(estimator =cls ,param_distributions =params ,cv =4 ,n_iter =5)
rs_result =rs.fit(xtr ,ytr)

In [4]:
# retrain
best_params =rs_result.best_params_
model =create_model(best_params['model__a_function'] ,best_params['model__h_unit'] ,best_params['model__l_rate'])
model_path ='model.keras'
model.save(model_path ,overwrite =True ,zipped=None)
epochs =100 
batch_size =32
hist =model.fit(xtr ,ytr ,batch_size =batch_size ,epochs =epochs ,validation_data =(xte ,yte) ,verbose =2)

In [5]:
# model evaluation 
Y =model.predict(xte) 
Yte =(Y > 0.5).astype(np.int32)
eva =[]
acc =accuracy_score(y_true =yte ,y_pred =Yte)
pre =precision_score(y_true =yte ,y_pred =Yte)
re =recall_score(y_true =yte ,y_pred =Yte)
f1 =f1_score(y_true =yte ,y_pred =Yte)
eva.append(acc)
eva.append(pre)
eva.append(re)
eva.append(f1)
eva

In [6]:
# evaluation plot
def font_config(family ,color ,weight ,size) : 
    font ={'family': family ,
        'color':  color ,
        'weight': weight ,
        'size': size 
        }
    return font
def plot_detail(plot ,scale) : 
    font_label =font_config('serif' ,'black' ,'normal' ,10)
    font_title =font_config('serif' ,'darkred' ,'bold' ,15)
    plt.xticks(ticks =np.arange(0 ,epochs+1 ,10))
    plt.yticks(ticks =np.arange(np.min(hist.history[plot]) ,np.max(hist.history[plot]) ,scale))
    plt.xlabel(xlabel ='epochs' ,labelpad =5 ,fontdict =font_label)
    plt.ylabel(ylabel =plot ,labelpad =5 ,fontdict =font_label)
    plt.title(label =f'{plot} plot' ,pad=5 ,fontdict =font_title)
    plt.legend()
fig =plt.figure(num =2 ,figsize =(12,10) ,dpi =100) 
ax1 =fig.add_subplot(2,2,1)
ax1.plot(hist.history['loss'] ,color ='blue' ,lw =3 ,ls ='-' ,label ='loss')
plot_detail('loss' ,0.1)
ax2 =fig.add_subplot(2,2,2)
ax2.plot(hist.history['val_loss'] ,color ='red' ,lw =3 ,ls ='-' ,label ='val_loss')
plot_detail('val_loss' ,0.1)
ax3 =fig.add_subplot(2,2,3)
ax3.plot(hist.history['binary_accuracy'] ,color ='teal' ,lw =3 ,ls ='-' ,label ='binary_accuracy')
plot_detail('binary_accuracy' ,0.1)
ax4 =fig.add_subplot(2,2,4)
plt.plot(hist.history['val_binary_accuracy'] ,color ='olive' ,lw =3 ,ls ='-' ,label ='val_binary_accuracy')
plot_detail('val_binary_accuracy' ,0.1)
plt.show()